In [ ]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 4.1 MB/s 
     |████████████████████████████████| 3.3 MB 21.1 MB/s 
     |████████████████████████████████| 636 kB 48.1 MB/s 
     |████████████████████████████████| 895 kB 50.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
import numpy as np
import pandas as pd
import transformers
from sklearn.model_selection import train_test_split
from transformers import AutoModel, BertTokenizerFast

In [ ]:
torch.cuda.is_available()

True

# Extracting Data and combining headline and content

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/My Drive
!ls

/content/drive/My Drive
 2019.csv				        EN_dic_fulldata_by_date.npy
 2019_large.csv				        EN_model_train.ipynb
'AMLH_assig_4 - 副本.ipynb'		        finbert
 analyst_ratings_processed.csv		        finbert_msft.h5
 analyst_small.csv			        new_0803
 BERT_huggingface_tutorial.ipynb	        NewsSentimentAnalysis-master
 CH_data_0805.csv			        nytimesapi.ipynb
 CH_data_2_label.csv			        Sentences_AllAgree.txt
 CH_data_3_label.csv			        small_data.csv
 chinese_news_0805.csv			        SourceHanSansCN-Regular.otf
 chinese_news.csv			        SourceHanSansCN-VF.otf
 chinese-text-classification.ipynb	       'Test 2.db'
 chinese-text-classification-using-bert.ipynb   Transformers_Demo.ipynb
 choice_news_ustime.csv			        Untitled0.ipynb
'Colab Notebooks'			        Untitled1.ipynb
 Confusion_matrix_nn.png		        Untitled2.ipynb
 data					        Untitled3.ipynb
 data_0802.csv				        us_equities_news_dataset.csv
 data_2_label.csv			        us_small.csv
 data_3_label.c

In [ ]:
df = pd.read_csv('CH_data_0805.csv')[['text','tag']]

df.dropna()
df.head()

,text,tag
0,《习近平关于严明党的纪律和规矩论述摘编》出版发行,2
1,以实际行动向党中央看齐 向高标准努力,2
2,【年终特稿】关键之年 改革挺进深水区,2
3,【2016 我们来了！】全国各地举行2016新年倒计时活动,2
4,【2016 我们来了！】世界各国多种形式喜迎2016年,2


,text,tag
0,接任欧盟轮值主席国 葡萄牙担子不轻,1
1,联大通过联合国2021年常规预算,1
2,亚马逊在华一审败诉 被判不得使用AWS名称并赔偿7646万元,1
3,俄罗斯通过“土耳其溪”管道向塞尔维亚和波黑供天然气,1
4,2020年全球宏观经济速览：遭遇疫情剧烈冲击后温和复苏,1
...,...,...
20577,在线教育资本大战冲击传统巨头 学大教育回归A股遭围剿,-1
20578,中澳自贸协定:“双刃效应”瞄准中国,-1
20579,欲摆脱“通道”角色 信托加码ABS主动管理,-1
20580,证监会急召券商投行赴京“吹风” IPO新规落地元月实施,-1


,text,tag
0,陆军领导机构火箭军战略支援部队成立大会在京举行 习近平向中国人民解放军陆军火箭军战略支援部队...,2
1,中央军委印发《关于深化国防和军队改革的意见》,2
2,《习近平关于严明党的纪律和规矩论述摘编》出版发行,2
3,以实际行动向党中央看齐 向高标准努力,2
4,【年终特稿】关键之年 改革挺进深水区,2
...,...,...
20733,部分反政府武装从伊德利卜撤重武器,1
20734,IMF下调今明两年世界经济增长预期,1
20735,乌克兰一军火库爆炸 上万人疏散,1
20736,联合国：全球控温“1.5度”更有益处,1


data saved


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df['text'], df['tag'],
                                                    stratify=df['tag'])

In [ ]:
x_train

21732           美股开盘：三大股指高开 　第九城市(NCTY.US)涨超30%
2366                         伊朗：俄制S－300防空导弹系统就位
1386                        《习近平谈治国理政》研讨会在布拉格举行
20559    鲍威尔：美联储没有考虑负利率 　经济下行风险显著 　需更多措施使经济走出困境
7589                        中国大洋38航次启航 “蛟龙”再探深海
                          ...                  
21753                      亚行：今明两年亚太经济增长强劲但增速放缓
329                            李克强应约同德国总理默克尔通电话
7481                             阿拉木图大冬会 中国队夺首金
13491                           中美联合侦破多起重特大毒品案件
8497                        张高丽将出席博鳌亚洲论坛2017年年会
Name: text, Length: 16485, dtype: object

# Preparing Data for Model

In [ ]:
bert = AutoModel.from_pretrained('bert-base-chinese')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
train_idx = x_train.dropna().index
test_idx = x_test.dropna().index

train_tokens = tokenizer.batch_encode_plus(x_train[train_idx].to_list(),
                                           max_length = 50,
                                           pad_to_max_length = True,
                                           truncation = True)
test_tokens = tokenizer.batch_encode_plus(x_test[test_idx].to_list(),
                                           max_length = 50,
                                           pad_to_max_length = True,
                                           truncation = True)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
train_seq = torch.tensor(train_tokens['input_ids'])
train_mask = torch.tensor(train_tokens['attention_mask'])
train_y = torch.tensor(y_train[train_idx].to_list())

test_seq = torch.tensor(test_tokens['input_ids'])
test_mask = torch.tensor(test_tokens['attention_mask'])
test_y = torch.tensor(y_test[test_idx].to_list())

In [ ]:
from torch.utils.data import TensorDataset, RandomSampler, DataLoader

In [ ]:
train_data = TensorDataset(train_seq, train_mask, train_y)
train_sampler = RandomSampler(train_data)
trainloader = DataLoader(train_data, 
                         sampler = train_sampler,
                         batch_size = 32)

test_data = TensorDataset(test_seq, test_mask, test_y)
test_sampler = RandomSampler(test_data)
testloader = DataLoader(test_data, 
                         sampler = test_sampler,
                         batch_size = 32)

In [ ]:
for param in bert.parameters():
    param.requires_grad = False

# Making our Model

In [ ]:
from torch import nn
from transformers import AdamW
import torch.nn.functional as F
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
class BertClassifier(nn.Module):
    def __init__(self, bert):
        super().__init__()
        self.bert = bert
        self.fc1 = nn.Linear(768,3)
    
    def forward(self, sent_id, mask):
        _ , cls_hs = self.bert(sent_id, attention_mask = mask,return_dict=False)
        return F.log_softmax(self.fc1(cls_hs), dim = 1)

In [ ]:
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = BertClassifier(bert)
model = model.cuda()

In [ ]:
optimizer = AdamW(model.parameters(), lr = 1e-5)

In [ ]:
class_weights = compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights

array([1.11960065, 1.56152316, 0.681931  ])

In [ ]:
weights = torch.tensor(class_weights, dtype = torch.float)
weights = weights.cuda()

criterion = nn.NLLLoss(weight = weights)

# Fine Tuning our model

In [ ]:
from tqdm.notebook import tqdm

AttributeError: ignored

In [ ]:
epochs = 20

for e in range(epochs):   
    train_loss = 0.0
    for batch in tqdm(trainloader):
        batch = [i.cuda() for i in batch]
        sent_id, masks, labels = batch

        optimizer.zero_grad()
        preds = model(sent_id, masks)
        loss = criterion(preds, labels)
        train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
    print(f'Epoch:{e+1}\t\tTraining Loss: {train_loss / len(trainloader)}')


Epoch:1		Training Loss: 0.8805470158194386



Epoch:2		Training Loss: 0.8729427742172581



Epoch:3		Training Loss: 0.8665596778078597



Epoch:4		Training Loss: 0.8606007092220839



Epoch:5		Training Loss: 0.855074275602666


...


Epoch:16		Training Loss: 0.8211299577193667



Epoch:17		Training Loss: 0.8191108578859374



Epoch:18		Training Loss: 0.8170999319747437



Epoch:19		Training Loss: 0.8158512835123742



Epoch:20		Training Loss: 0.8140943715738695


In [ ]:
pred_label = []
true_label = []
for batch in tqdm(testloader):
    batch = [i.cuda() for i in batch]
    sent_id, masks, labels = batch

    preds = model(sent_id, masks)
    pred_label.extend(torch.argmax(preds, axis = 1).cpu())
    true_label.extend(labels.cpu())

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
confusion_matrix(true_label, pred_label)

array([[ 874,  475,  288],
       [ 173,  937,   63],
       [ 459,  564, 1663]])

In [ ]:
print(classification_report(true_label, pred_label))

              precision    recall  f1-score   support

           0       0.58      0.53      0.56      1637
           1       0.47      0.80      0.60      1173
           2       0.83      0.62      0.71      2686

    accuracy                           0.63      5496
   macro avg       0.63      0.65      0.62      5496
weighted avg       0.68      0.63      0.64      5496

